In [1]:
# importing of required packages
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import train_test_split

lemma = WordNetLemmatizer()

In [18]:
# function to merging name,associated software and Description 
def creating_training_data(df,label):
    df.fillna('',inplace=True)
    text=[]
    names=list(df['Name'])
    col = ['Name','Associated Software','Description']
    initial = True
    for i in col:
        if initial:
            a= df[i].str.lower()
            initial = False
        else:
            a = a+' '+df[i].str.lower()
            
    text.append(a)
    result = pd.DataFrame({'Name':names,'Description':text[0],'label':label})
    return result

In [19]:
# importing threat_actor,mlaware and tool data and giving labels
threat_actor = pd.read_csv(r'data/threat_actors.csv')
malware = pd.read_csv(r'data/malware_raw.csv')
tool = pd.read_csv(r'data/tool_raw.csv')
# labelling  and calling the above merging Name,associated software,description
threat_actor['label'] = 0
malware_clean = creating_training_data(malware,label=1)
tool_clean = creating_training_data(tool,label=2)
# segregating the training  data set from the above extracted data
data = threat_actor[['Name','Description','label']]
data = data.append(malware_clean[['Name','Description','label']])
data = data.append(tool_clean[['Name','Description','label']])

In [22]:
data.to_csv('data/training_data_malware_all.csv',index=False)

In [7]:
# shape of the data is
print('dimensions of the data is',data.shape)
# sample of data description and its label
print('\n sample_data')
data.sample(5)


dimensions of the data is (508, 2)

 sample_data


,Description,label
7,allwinner allwinner is a company that supplie...,1
2,adbupd adbupd is a backdoor used by platinum ...,1
243,rarstone rarstone is malware used by the naik...,1
72,dok retefe dok steals banking information thro...,1
21,ipconfig ipconfig.exe ipconfig is a windows ut...,2


In [6]:
data

,Description,label
0,G0018 admin@338 is a China-based cyber threat ...,0
1,G0006 APT1 is a Chinese threat group that has ...,0
2,G0005 APT12 is a threat group that has been at...,0
3,G0023 APT16 is a China-based threat group that...,0
4,G0025 APT17 is a China-based threat group that...,0
...,...,...
55,uacme uacme is an open source assessment tool...,2
56,windows credential editor wce windows credenti...,2
57,"winexe winexe is a lightweight, open source t...",2
58,xbot xbot is an android malware family that w...,2


In [7]:
# checking for the null values in the data
print('Null values in data')
print(data.isna().sum())
print('\ncount of target labels in data')
print(data.label.value_counts())

Null values in data
Description    0
label          0
dtype: int64

count of target labels in data
1    354
0     94
2     60
Name: label, dtype: int64


In [8]:
# converting description to lower case 
data['Description'] = data['Description'].str.lower()

In [9]:
#function to preprocess th data
def text_preprocessing(df):
    stop_words = stopwords.words("english")+list(punctuation) # segregation stopwords and punctuations
    tokens=[]
    for index,row in df.iterrows():
        text = row['Description']
        words = [w for sent in nltk.sent_tokenize(str(text)) for w in nltk.word_tokenize(sent)] #applying tokenization
        words = [w for w in words if w not in stop_words and w.isalnum()] # removing stopwords and only alphanumeric
        words = [w for w in words if len(w)>1] #removing if length is less than 1
        words = [lemma.lemmatize(w,'v') for w in words ] # applying lemmatization
        df['Description'][index] = ' '.join(words)
        tokens.extend(words)
        
    term_frequency = Counter(tokens)
    return df,term_frequency
        

In [10]:
data,term_freq = text_preprocessing(data) # text_preprocessing
# splitting of data in train size 0.7 and test size 0.3 ratio
X_train, X_test, y_train, y_test = train_test_split(data['Description'], data['label'],test_size=0.30, random_state = 123)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
 
docs=list(data['Description'])
word_count_vector=cv.fit_transform(docs)

In [12]:
cv.vocabulary_

{'arp': 103,
 'display': 342,
 'information': 624,
 'system': 1347,
 'address': 39,
 'resolution': 1151,
 'protocol': 1067,
 'cache': 171,
 'use': 1458,
 'schedule': 1192,
 'task': 1363,
 'run': 1180,
 'specify': 1285,
 'date': 288,
 'time': 1392,
 'bitsadmin': 151,
 'command': 211,
 'line': 746,
 'tool': 1398,
 'create': 261,
 'manage': 791,
 'bits': 150,
 'job': 684,
 'cachedump': 172,
 'program': 1061,
 'extract': 455,
 'password': 977,
 'hash': 566,
 'registry': 1130,
 'certutil': 191,
 'utility': 1465,
 'obtain': 919,
 'certificate': 190,
 'authority': 123,
 'configure': 232,
 'service': 1229,
 'cmd': 205,
 'windows': 1517,
 'interpreter': 651,
 'interact': 645,
 'systems': 1349,
 'execute': 443,
 'process': 1058,
 'utilities': 1464,
 'contain': 241,
 'native': 873,
 'functionality': 513,
 'perform': 985,
 'many': 798,
 'operations': 945,
 'include': 615,
 'list': 751,
 'file': 470,
 'directory': 337,
 'dir': 333,
 'delete': 300,
 'del': 299,
 'copy': 247,
 'cobalt': 206,
 'strike

In [13]:
df=pd.DataFrame(dict(term_freq).items())
#df.to_csv('term_freq_model.csv')

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer 
 
# settings that you use for count vectorizer will go here
tfidf_vectorizer=TfidfVectorizer(use_idf=True,min_df=1, ngram_range=(1,3))

tfidf_vectorizer_vectors=tfidf_vectorizer.fit(X_train)
X_train_tfidf = tfidf_vectorizer_vectors.transform(X_train)
X_test_tfidf = tfidf_vectorizer_vectors.transform(X_test)

clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)
accuracy_score(y_test, y_pred)

0.6797385620915033

In [15]:
#checking sparsity
from numpy import count_nonzero
sparsity = 1.0 - ( count_nonzero(X_train_tfidf.toarray()) / float(X_train_tfidf.toarray().size) )
print(sparsity)

0.9946780192598119


In [17]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print('confusion_matrix %s') 
print(confusion_matrix(y_pred, y_test))

accuracy 0.6928104575163399
confusion_matrix %s
[[  0   0   0]
 [ 29 106  18]
 [  0   0   0]]


In [20]:
nb.predict(['hello hai'])

array([1], dtype=int64)

In [14]:
nb.predict_proba(['4H RAT is  that has been used by Putter Panda since at least 2007'])

array([[0.05876865, 0.9074961 , 0.03373525]])

In [17]:
sg = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=420, max_iter=5, tol=None)),
              ])
sg.fit(X_train, y_train)
y_pred = sg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print('confusion_matrix %s') 
print(confusion_matrix(y_pred, y_test))

accuracy 0.5294117647058824
confusion_matrix %s
[[ 2 17  3]
 [23 79 15]
 [ 4 10  0]]


In [ ]:
nb.predict_proba(X_test)

In [26]:
import pandas as pd
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
import stringdist


lemma = WordNetLemmatizer()

df=pd.read_csv(r'D:\MY STUFF\rhym\data/concatenated_ta.csv',index_col='name')
df.reset_index(inplace=True)

In [27]:
text="""Since 2009, HIDDEN COBRA actors have leveraged their capabilities to target and compromise a range of victims; some intrusions have resulted in the exfiltration of data while others have been disruptive in nature. Commercial reporting has referred to this activity as Lazarus Group and Guardians of Peace. Tools and capabilities used by HIDDEN COBRA actors include DDoS botnets, keyloggers, remote access tools (RATs), and wiper malware. Variants of malware and tools used by HIDDEN COBRA actors include Destover, Duuzer, and Hangman."""

In [30]:
stop_words = stopwords.words("english")+list(punctuation)
words = [w.lower() for sent in nltk.sent_tokenize(str(text)) for w in nltk.word_tokenize(sent)] #applying tokenization
words = [w for w in words if w not in stop_words and w.isalnum()] # removing stopwords and only alphanumeric
words = [w for w in words if len(w)>1] #removing if length is less than 1
tokens = [lemma.lemmatize(w,'v')  for w in words]

In [36]:
thresh=0.5
scoring=dict()
for index,row in df.iterrows():
    initial=True
    for i in tokens:
        score=0
        for j in row['keywords'].split(' '):
            dist = stringdist.levenshtein_norm(i,j)
            if dist < 0.30:
                print(i,j)
                score+=dist
    print(row['name'],score)
#             if dist > thresh:
#                 if initial:
#                     scoring[row['name']]=1
#                     initial = False
#                 else:
#                     scoring[row['name']] = scoring[row['name']]+1


since line
actors actors
actors backdoors
capabilities vulnerabilities
target targeted
compromise compromised
victims victims
exfiltration exploitation
exfiltration configuration
data data
nature capture
report port
group groups
group group
tool tools
capabilities vulnerabilities
use used
use user
actors actors
actors backdoors
remote remote
remote removal
access process
tool tools
rat rats
malware malware
malware malware
tool tools
use used
use user
actors actors
actors backdoors
destover discovery
destover deliver
admin@338 0
since line
capabilities utilities
target mapiget
compromise compress
compromise compromise
compromise compressed
compromise compromised
victims victim
result seasalt
exfiltration registration
exfiltration configuration
exfiltration liberation
exfiltration escalation
data data
nature capture
report support
report records
refer reader
group group
group groups
guardians mandiant
tool toolkit
tool tools
capabilities utilities
use used
use use
use user
use uses
botne

actors sectors
include encode
remote remote
remote removal
access process
access access
tool tools
tool tool
wiper wipe
malware malware
variants variety
variants various
malware malware
tool tools
tool tool
use used
use user
use users
hide video
actors sectors
include encode
destover discovery
destover developer
apt33 0
since line
leverage leverages
capabilities capable
target torrent
target harvest
target targeted
compromise compromise
compromise compromises
compromise compressed
compromise compromised
range range
victims victim
victims victims
exfiltration exploitation
exfiltration exfiltration
data data
others northern
disruptive destructive
nature capture
nature captures
report reboot
report port
report record
refer reaper
refer refer
activity activity
activity utility
activity active
lazarus lazarus
group group
group group123
group groups
guardians variant
capabilities capable
use user
use used
use uses
use use
use users
include including
remote remote
access process
access access

exfiltration configuration
nature capture
report imports
refer peer
activity activity
activity active
group drop
group group
peace pack
use used
use users
use user
actors backdoors
actors actors
keyloggers keylogger
remote removable
access address
access process
rat rar
malware malware
malware malware
use used
use users
use user
actors backdoors
actors actors
destover discovery
darkhotel 0
since line
2009 g0079
hide hidden
leverage leverages
leverage leveraged
target targeted
victims victims
intrusions institutions
exfiltration escalation
exfiltration exploitation
exfiltration configuration
data data
nature capture
report support
report port
group group
tool tools
tool tool
use user
use users
use used
use use
hide hidden
remote remote
remote removal
access process
access access
tool tools
tool tool
rat rar
malware malware
malware malware
tool tools
tool tool
use user
use users
use used
use use
hide hidden
destover discovery
darkhydrus 0
since vine
since line
2009 g0009
hide hidden
hide

leverage leverage
target targeted
victims victim
victims victims
exfiltration configuration
data data
nature capture
nature features
report port
activity deactivate
group group
guardians variant
tool tools
tool tool
use use
use user
use used
use users
hide video
hide hidden
include including
remote remote
remote removable
access process
access access
tool tools
tool tool
rat njrat
malware malware
variants variant
malware malware
tool tools
tool tool
use use
use user
use used
use users
hide video
hide hidden
include including
destover discovery
gorgon group 0
since line
hide video
capabilities capable
target targeted
victims victim
victims victims
exfiltration configuration
data data
nature capture
report port
lazarus layers
group group5
group group
tool tools
capabilities capable
use used
use user
hide video
include including
remote remote
remote remotely
remote removable
access access
tool tools
rat njrat
rat rats
rat rat
malware malware
malware malware
tool tools
use used
use user
hi

KeyboardInterrupt: 

In [13]:
sorted(scoring.items(), key=lambda x: x[1], reverse=True)

[('lazarus group', 28),
 ('patchwork', 21),
 ('threat group-3390', 21),
 ('night dragon', 19),
 ('apt41', 18),
 ('apt28', 17),
 ('axiom', 17),
 ('apt38', 16),
 ('ke3chang', 16),
 ('apt18', 15),
 ('apt37', 15),
 ('bronze butler', 15),
 ('cobalt group', 15),
 ('dragonfly 2.0', 15),
 ('menupass', 15),
 ('apt32', 14),
 ('oilrig', 14),
 ('temp.veles', 14),
 ('apt3', 13),
 ('fin6', 13),
 ('gorgon group', 13),
 ('carbanak', 12),
 ('darkhydrus', 12),
 ('molerats', 12),
 ('apt29', 11),
 ('apt33', 11),
 ('fin5', 11),
 ('magic hound', 11),
 ('elderwood', 10),
 ('fin8', 10),
 ('muddywater', 10),
 ('the white company', 10),
 ('admin@338', 9),
 ('apt19', 9),
 ('copykittens', 9),
 ('darkhotel', 9),
 ('deep panda', 9),
 ('fin7', 9),
 ('platinum', 9),
 ('strider', 9),
 ('tropic trooper', 9),
 ('apt1', 8),
 ('cleaver', 8),
 ('group5', 8),
 ('leafminer', 8),
 ('leviathan', 8),
 ('putter panda', 8),
 ('sowbug', 8),
 ('stolen pencil', 8),
 ('ta505', 8),
 ('threat group-1314', 8),
 ('apt39', 7),
 ('dark car